### Importing packages

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf

### Load data

In [ ]:
df_labels = pd.read_csv("/kaggle/input/dog-breed-identification/labels.csv")
df_labels['id'] = df_labels['id'] + '.jpg'
df_labels.head()

### Splitting data

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_validation = train_test_split(df_labels, test_size=0.2, shuffle=True)

### Verifying data and generating images for testing

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
        rescale=1./255)

training_set = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory='/kaggle/input/dog-breed-identification/train',
    x_col="id",
    y_col="breed",
    target_size=(150, 150),
    class_mode="categorical",
    batch_size=128
)

validation_set = validation_datagen.flow_from_dataframe(
    dataframe=df_validation,
    directory='/kaggle/input/dog-breed-identification/train',
    x_col="id",
    y_col="breed",
    target_size=(150, 150),
    class_mode="categorical",
    batch_size=128
)

In [ ]:
classes = df_labels['breed'].unique().shape[0]
print("Total breeds: ", classes)

### Model

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()


model.add(Conv2D(32, (3, 3),input_shape=(150,150,3), activation = 'relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3), activation = 'relu'))

model.add(Flatten())
model.add(Dropout(0.25))

model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=classes, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.fit(x = training_set, validation_data = validation_set, epochs = 20)

In [ ]:
test_datagen = ImageDataGenerator ( 
    rescale=1/255.
)

test_set = test_datagen.flow_from_directory (
    '/kaggle/input/dog-breed-identification',
    target_size = (150,150),
    batch_size = 128,
    classes=['test']
)

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
df_submission = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')
df_submission

In [ ]:
import re
file_list = test_set.filenames
id_list = []
for name in file_list:
    m = re.sub('test/', '', name)
    m = re.sub('.jpg', '', m)
    id_list.append(m)

In [ ]:
df_submission['id'] = id_list
df_submission.iloc[:,1:] = y_pred
df_submission

In [ ]:
final_df = df_submission.set_index('id')
final_df.to_csv('Submission.csv')